In [1]:
from app import db
import numpy as np
from app.models import (
    Bracket,
    Match,
    Round,
    Tournament,
    User,
    bracket_entrants 
)
from sqlalchemy.sql.expression import func, select


In [2]:
db.session.close()

In [3]:
def purge(Model):
    [db.session.delete(row) for row in Model.query.all()]

In [4]:
# purge all tables first
for Model in [Bracket, Match, Round, Tournament, User]:
    purge(Model)


IntegrityError: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(sqlite3.IntegrityError) NOT NULL constraint failed: round.bracket_id
[SQL: UPDATE round SET bracket_id=? WHERE round.id = ?]
[parameters: (None, 1)]
(Background on this error at: http://sqlalche.me/e/gkpj)

In [5]:

db.session.commit()


In [6]:
# Fall Charity LAN 2018 Melee Singles users
users = [
    User(username='TPN', email='TPN@example.com'.lower()), 
    User(username='AngryFalco', email='AngryFalco@example.com'.lower()), 
    User(username='Sunrisebanana', email='Sunrisebanana@example.com'.lower()), 
    User(username='Ptolemy', email='Ptolemy@example.com'.lower()), 
    User(username='Vik', email='Vik@example.com'.lower()), 
    User(username='Kevin', email='Kevin@example.com'.lower()), 
    User(username='Spaceghost', email='Spaceghost@example.com'.lower()), 
    User(username='Burnaby', email='Burnaby@example.com'.lower()), 
]

[db.session.add(u) for u in users]
db.session.commit()

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: user.username
[SQL: INSERT INTO user (username, email, password_hash, about_me, last_seen) VALUES (?, ?, ?, ?, ?)]
[parameters: ('TPN', 'tpn@example.com', None, None, '2020-02-05 13:45:46.678584')]
(Background on this error at: http://sqlalche.me/e/gkpj)

In [7]:
users = User.query.all()
np.random.shuffle(users)

In [8]:

# set up tournament with all users
tournament_name = 'Fall Charity LAN 2018 Melee Singles'
t = \
    Tournament(
        n_entrants = len(users), 
        name=tournament_name,
        organizer_id=User.query.order_by(func.random()).first().id
    )
db.session.add(t)
db.session.commit()

In [9]:
t

<Tournament 1>

In [10]:
b = \
    Bracket(
        bracket_type='Winners',
        users=[u for u in User.query.order_by(func.random()).limit(t.n_entrants).all()],
        tournament=t,
    )

db.session.add(b)
db.session.commit()

In [11]:
# place mock users in rounds

In [12]:
r = \
    Round(
        number=1,
        winners=True,
        bracket=b
    )

db.session.add(r)
db.session.commit()